# Building LLM applications: Notebook 04

# ReAct

**WARNING**: We are using an "old fasion" implementation based on `AgentExecutor` for illustration pourposes only.
A more "modern" approach is to use LangGraph

## Initialize

In [1]:
import os
import dotenv

from langchain_core.agents import AgentAction
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser

from langchain_ollama import ChatOllama

In [2]:
MODEL = 'phi4'

In [3]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: http://kqrw311-g5-12xlarge-a.img.astrazeneca.net:8080


## ReAct Agent

### Step 1: Create tools (`str` parameters)

In [4]:
@tool
def double_of(n):
    """Double of a number"""
    print(f"Double of (type: {type(n)}) {n}")
    n = int(n)
    print(f"Double of {n}")
    return 2 * n


@tool
def factorial_of(n):
    """Factorial of a number"""
    print(f"Factorial of (type: {type(n)}) {n}")
    n = int(n)
    fact = 1
    for i in range(1, n+1):
        fact *= i
    return fact


In [5]:
tools = [double_of, factorial_of]
tools_map = {t.name.lower(): t for t in tools}
tools_map

{'double_of': StructuredTool(name='double_of', description='Double of a number', args_schema=<class 'langchain_core.utils.pydantic.double_of'>, func=<function double_of at 0x10c441440>),
 'factorial_of': StructuredTool(name='factorial_of', description='Factorial of a number', args_schema=<class 'langchain_core.utils.pydantic.factorial_of'>, func=<function factorial_of at 0x10c4413a0>)}

### Step 2: Create a ReAct Agent prompt

In [6]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt object from the string template
prompt_template = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names'], input_types={}, partial_variables={}, template='Use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}\n')

### Step 3: We need to add the `intermediate_steps` to the `agent_scratchpad`

In [7]:
def format_log_to_str(
    intermediate_steps: list[tuple[AgentAction, str]],
    observation_prefix: str = "Observation: ",
    llm_prefix: str = "Thought: ",
) -> str:
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts = ""
    for action, observation in intermediate_steps:
        thoughts += action.log
        thoughts += f"\n{observation_prefix}{observation}\n{llm_prefix}"
    return thoughts


### Step 4: Create the `AgentExecutor`

In [8]:
agent = create_tool_calling_agent(llm, tools, prompt)

llm_with_stop = llm.bind(stop=["\nObservation"])
agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
    )
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)


agent_executor = AgentExecutor(agent=agent, tools=tools)

### Run it

In [9]:
query = "What is the factorial of 10?"
tool_names = ','.join([t.name for t in tools])

agent_executor.invoke({'input': query, 'tool_names': tool_names})

Factorial of (type: <class 'str'>) 10

Factorial of (type: <class 'str'>) 10

Factorial of (type: <class 'str'>) 10

Factorial of (type: <class 'str'>) 10



ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: I now know the final answer.

Final Answer: The factorial of 10 is 3,628,800.
Here's a breakdown of how this result was achieved:

Question: What is the factorial of 10?

Thought: To find the factorial of 10, I need to calculate \(10!\), which is the product of all positive integers from 1 to 10.

Action: factorial_of

Action Input: 10

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 